In [22]:
import numpy as np
import pandas as pd
import pickle
import math
import pandas as pd
from pandas import HDFStore 
import argparse

###################################################################################
#location
node_ids_filename = 'data/node_locate.txt'
with open(node_ids_filename) as f:
    _node_ids = f.read().strip()
    _node_ids = _node_ids.replace('\n', ' ')
    _node_ids = _node_ids.split(' ')

node_id = []
node_ids = []
node_name = []
node_loc = []
for i in range(len(_node_ids)):
    if(_node_ids[i] == ''):
        continue
    node_id.append(_node_ids[i])

for i in range(len(node_id)):
    if(i <= 4):
        continue
    if(node_id[i] == 'C'):
        continue
    node_ids.append(node_id[i])
for i in range(len(node_ids)):
    if(i % 4 == 0):
        line = []
        node_name.append(node_ids[i])
        line.append(node_ids[i+1])
        line.append(node_ids[i+2])
        line.append(node_ids[i+3])
        node_loc.append(line)
node_loc=np.array(node_loc).astype('float32')
print("node_loc",node_loc.shape)

###################################################################################
#save to npy file
np.save('data/node_signal', _node_signal)
np.save('data/node_loc', node_loc)

_node_signal (135, 106, 7)
node_loc (106, 3)
-118.45 139.457
-31.3102 21.6857
-563.119 91.0359
-0.0160272 0.000982589
-0.0147581 0.0163793
-0.00474041 0.0034676
0.0 0.0


In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import pandas as pd
from pandas import HDFStore 
import argparse
import matplotlib.pyplot as plt

####################################################################################
node_loc=np.load('data/node_loc.npy')
n_nodes=node_loc.shape[0]
node_id = [str(i) for i in range(1,1+n_nodes)]
node_id_to_ind = {}
for i, node in enumerate(node_id, 1):
    node_id_to_ind[node] = i
num_node = len(node_id)

#element 1################
node_link_filename = 'data/element1.txt'
link_mx = np.zeros((num_node, num_node), dtype=np.float32)
with open(node_link_filename) as f:
    _node_link = f.read().strip()
    _node_link = _node_link.replace('\n', ' ')
    _node_link = _node_link.split(' ')

node_link = []
for i in range(len(_node_link)):
    if(_node_link[i] == ''):
        continue
    node_link.append(_node_link[i])

element_id = []

node_link = np.reshape(node_link, (-1, 6))
node_link = node_link.astype('int32')

link_mx = np.zeros((num_node, num_node), dtype=np.float32)
for i in range(39):
    link_mx[node_link[i][2]-1][node_link[i][3]-1] = node_link[i][1]
    link_mx[node_link[i][3]-1][node_link[i][2]-1] = node_link[i][1]
link_mx1_1=link_mx

link_mx = np.zeros((num_node, num_node), dtype=np.float32)
for i in range(39,node_link.shape[0]):
    link_mx[node_link[i][2]-1][node_link[i][3]-1] = node_link[i][1]
    link_mx[node_link[i][3]-1][node_link[i][2]-1] = node_link[i][1]    
link_mx1_2=link_mx

#element 2################
node_link_filename = 'data/element2.txt'
link_mx = np.zeros((num_node, num_node), dtype=np.float32)
with open(node_link_filename) as f:
    _node_link = f.read().strip()
    _node_link = _node_link.replace('\n', ' ')
    _node_link = _node_link.split(' ')

node_link = []
for i in range(len(_node_link)):
    if(_node_link[i] == ''):
        continue
    node_link.append(_node_link[i])

element_id = []
node_link=[i for i in node_link if i.isdigit()]
node_link = np.reshape(node_link, (-1, 5))
node_link = node_link.astype('int32')
for i in range(node_link.shape[0]):
    link_mx[node_link[i][1]-1][node_link[i][2]-1] = node_link[i][1]
    link_mx[node_link[i][2]-1][node_link[i][1]-1] = node_link[i][1]
link_mx2=link_mx

#element 3################
node_link_filename = 'data/element3.txt'
link_mx = np.zeros((num_node, num_node), dtype=np.float32)
with open(node_link_filename) as f:
    _node_link = f.read().strip()
    _node_link = _node_link.replace('\n', ' ')
    _node_link = _node_link.split(' ')

node_link = []
for i in range(len(_node_link)):
    if(_node_link[i] == ''):
        continue
    node_link.append(_node_link[i])

element_id = []

node_link = np.reshape(node_link, (-1, 6))
node_link = node_link.astype('int32')
for i in range(node_link.shape[0]):
    link_mx[node_link[i][2]-1][node_link[i][3]-1] = node_link[i][1]
    link_mx[node_link[i][3]-1][node_link[i][2]-1] = node_link[i][1]
link_mx3=link_mx

####################################################################################
link_mx=link_mx1_1+link_mx1_2+link_mx2+link_mx3
#np.save('data/link_mx', link_mx)
np.sum(link_mx1_1!=0),np.sum(link_mx1_2!=0),np.sum(link_mx2!=0),np.sum(link_mx3!=0)


####################################################################################
#adj matrix
#link_mx=np.load('data/link_mx.npy')
link_mx0=link_mx1_1+link_mx1_2+link_mx2+link_mx3
link_mx0=link_mx0+np.eye(106)
link_mx0[link_mx0!=0]=1

link_mx1_1=link_mx1_1+np.eye(106)#cross beam
link_mx1_1[link_mx1_1!=0]=1

link_mx1_2=link_mx1_2+np.eye(106)#girder
link_mx1_2[link_mx1_2!=0]=1

link_mx2=link_mx2+np.eye(106)#cable
link_mx2[link_mx2!=0]=1

link_mx3=link_mx3+np.eye(106)#tower
link_mx3[link_mx3!=0]=1

####################################################################################
#getting distance matrix
node_loc=np.load('data/node_loc.npy')
num_node=len(node_loc)
n_nodes=node_loc.shape[0]
dist_mx = np.zeros((num_node, num_node), dtype=np.float32)
dist_mx[:] = np.inf
for i in range(num_node):
    for j in range(num_node):
        x = float(node_loc[i][0]) - float(node_loc[j][0])
        y = float(node_loc[i][1]) - float(node_loc[j][1])
        z = float(node_loc[i][2]) - float(node_loc[j][2])
        x = math.pow(x,2)
        y = math.pow(y,2)
        z = math.pow(z,2)
        dis = math.sqrt(x+y+z)
        dist_mx[i, j] = dis
distances = dist_mx[~np.isinf(dist_mx)].flatten()
std = distances.std()
adj_mx = np.exp(-np.square(dist_mx / std))

adj_mx0=adj_mx*link_mx0
print(adj_mx)

################################################
import collections
print(collections.Counter(adj_mx0.flatten()))

#cross beam-#girder-#cable-#tower
type_e=np.array([200000,200000,15800,200000])#type_e=np.array([210000,180000,140000,200000])
type_e=(type_e)/np.max(type_e)
print(type_e,type_e.std(),adj_mx0.std())

link_mx1_1_e=link_mx1_1-np.eye(106)
link_mx1_1_e=link_mx1_1_e*type_e[0]#cross beam

link_mx1_2_e=link_mx1_2-np.eye(106)
link_mx1_2_e=link_mx1_2_e*type_e[1]#girder

link_mx2_e=link_mx2-np.eye(106)
link_mx2_e=link_mx2_e*type_e[2]#cable

link_mx3_e=link_mx3-np.eye(106)
link_mx3_e=link_mx3_e*type_e[3]#tower

adj_mx_e=link_mx1_1_e+link_mx1_2_e+link_mx2_e+link_mx3_e
print(adj_mx_e)

print(collections.Counter(adj_mx_e.flatten()))

[[1.0000000e+00 9.8702234e-01 9.4909120e-01 ... 1.3570749e-05
  1.2861080e-05 1.2111769e-05]
 [9.8702234e-01 1.0000000e+00 9.8702234e-01 ... 2.8573637e-05
  2.7079383e-05 2.5501660e-05]
 [9.4909120e-01 9.8702234e-01 1.0000000e+00 ... 5.8611240e-05
  5.5546225e-05 5.2309995e-05]
 ...
 [1.3570749e-05 2.8573637e-05 5.8611240e-05 ... 1.0000000e+00
  9.9684554e-01 9.8744172e-01]
 [1.2861080e-05 2.7079383e-05 5.5546225e-05 ... 9.9684554e-01
  1.0000000e+00 9.9684554e-01]
 [1.2111769e-05 2.5501660e-05 5.2309995e-05 ... 9.8744172e-01
  9.9684554e-01 1.0000000e+00]]
Counter({0.0: 10760, 0.987022340297699: 152, 1.0: 106, 0.9804471135139465: 82, 0.996845543384552: 32, 0.2530885636806488: 16, 0.40820610523223877: 16, 0.5893293023109436: 16, 0.7615673542022705: 16, 0.8809071779251099: 16, 0.9719653725624084: 16, 0.965503990650177: 8})
[1.    1.    0.079 1.   ] 0.39880469844273403 0.1889514214168133
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 

In [9]:
import collections
print(collections.Counter(link_mx0.flatten()))
np.save('data_sage/sensor_graph/adj_wself',link_mx0)

Counter({0.0: 10760, 1.0: 476})


In [5]:
adj_mx=[adj_mx0,adj_mx_e]
print(np.sum(adj_mx0==0)/(106*106))
print(np.sum(adj_mx_e==0)/(106*106))
########################################
output_pkl_filename='data/sensor_graph/adj_mx_type_e.pkl'
########################################
with open(output_pkl_filename, 'wb') as f:
    pickle.dump([[], [], adj_mx], f, protocol=2)

0.9576361694553222
0.9670701317194731


In [2]:
#####area
#cross beam
w=165.10;d=525.78;wt=8.89;ft=11.43;cb_a=(w*d)-(w-wt)*(d-2*ft);print(cb_a)#8245.14480000001
#girder
b=1000;h=1000;t=50;g_a=g_a=(b*h)-((b-t)*(h-t));print(g_a)#97500
#cable
c_a=3848.45#0
#tower
b=1000;h=2000;t=50;t_a=(b*h)-((b-t)*(h-t));print(t_a)#147500

#cross beam-#girder-#cable-#tower
type_a=np.array([cb_a,g_a,c_a,t_a])#type_e=np.array([210000,180000,140000,200000])
type_a=(type_a)/np.max(type_a)
print(type_a,type_a.std())

link_mx1_1_a=link_mx1_1-np.eye(106)
link_mx1_1_a=link_mx1_1_a*type_a[0]#cross beam

link_mx1_2_a=link_mx1_2-np.eye(106)
link_mx1_2_a=link_mx1_2_a*type_a[1]#girder

link_mx2_a=link_mx2-np.eye(106)
link_mx2_a=link_mx2_a*type_a[2]#cable

link_mx3_a=link_mx3-np.eye(106)
link_mx3_a=link_mx3_a*type_a[3]#tower

adj_mx_a=link_mx1_1_a+link_mx1_2_a+link_mx2_a+link_mx3_a
print(collections.Counter(adj_mx_a.flatten()))

with open('data/sensor_graph/adj_mx_type_a.pkl', 'wb') as f:
    pickle.dump(adj_mx_a, f, protocol=2)

8245.14480000001
97500
147500
[0.05589929 0.66101695 0.02609119 1.        ] 0.41268334371753873
Counter({0.0: 10866, 0.6610169491525424: 152, 0.026091186440677965: 80, 0.05589928677966108: 78, 1.0: 60})
